In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

# replacing spaces with underscores:
available_classes = [cname.replace(' ', '_') for cname in available_classes]

In [3]:
# having it reverse for plot:
available_classes_r = available_classes[::-1]

In [4]:
available_models

['ViT-bigG-14-CLIPA-336',
 'RN50x64',
 'EVA02-E-14-plus',
 'ViT-H-14-378-quickgelu',
 'RN101',
 'ViT-B_32',
 'ViT-SO400M-14-SigLIP-384',
 'ViT-L_14']

In [5]:
available_classes

['ground',
 'concrete',
 'asphalt',
 'concrete_plates',
 'grass',
 'paving_stones',
 'sett']

In [6]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [7]:
{model_name: len(modelwise_dfs[model_name]) for model_name in modelwise_dfs}

{'RN50x64': 2619,
 'ViT-bigG-14-CLIPA-336': 2619,
 'ViT-L_14': 2619,
 'ViT-H-14-378-quickgelu': 2619,
 'ViT-SO400M-14-SigLIP-384': 2619,
 'RN101': 2619,
 'ViT-B_32': 2619,
 'EVA02-E-14-plus': 2619}

In [8]:
merged_df = pd.concat(all_dfs)

In [9]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ')
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [10]:
merged_pred

,asphalt,concrete,concrete_plates,grass,ground,paving_stones,sett,prediction,class
image_name,,,,,,,,,
00098db7c6cd6ee36d590ff0d5db7ad1.png,0.213754,3.985507,0.961183,0.019217,0.534829,1.280490,0.005109,concrete,concrete
00157343508e08f1dbc0efc1f8066f3d.png,0.601964,0.216773,0.454248,0.994805,2.932286,1.649854,0.149998,ground,ground
003a08eff81fe589f95102bf2ce1ecf5.png,0.007631,0.002122,0.013925,5.955048,0.965288,0.043469,0.012785,grass,grass
005b94f55127ae460c0741eac3a89b25.png,4.938634,0.277412,1.465858,0.012804,0.181552,0.108702,0.015524,asphalt,asphalt
006d04cdfa79fac8f108a0a1cd34d0f8.png,0.018306,0.004128,0.016005,6.087209,0.791766,0.057336,0.025230,grass,grass
...,...,...,...,...,...,...,...,...,...
ffbd835981460b0e1b7825681c7b9449.png,0.544958,0.495025,0.475408,0.098204,0.890578,3.950697,0.545128,paving_stones,concrete_plates
ffd0ab8395fc05d8d14f3cfe8a6a6b4a.png,0.600495,0.556082,0.496194,0.633679,3.802484,0.742776,0.168145,ground,ground
ffd4bc27fe750ca5fedac61da4f7a397.png,0.428585,0.148436,0.752849,0.359114,3.330605,1.677660,0.302741,ground,ground


In [11]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [12]:
c_mat

array([[ 0.   , 84.104,  0.289,  0.578,  1.156,  2.023, 11.85 ],
       [ 0.   , 97.885,  0.   ,  0.   ,  1.208,  0.302,  0.604],
       [ 0.   ,  0.   , 98.904,  0.   ,  0.   ,  0.   ,  1.096],
       [ 0.   , 49.714,  2.857, 15.429,  2.857,  9.143, 20.   ],
       [ 1.325,  0.   ,  5.96 ,  2.649, 54.084, 22.517, 13.466],
       [ 2.332,  2.332,  2.073,  4.404,  7.513, 72.539,  8.808],
       [ 0.   ,  0.212,  0.   ,  0.   ,  1.059,  0.   , 98.729]])

In [13]:
# sanity check on labels:
print(merged_pred[class_label].unique())

['concrete' 'ground' 'grass' 'asphalt' 'sett' 'paving_stones'
 'concrete_plates']


In [14]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [15]:
n_rows = int(ceil(len(available_models)/2))

In [16]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(true_labels,predicted_labels,labels=available_classes, normalize='true') * 100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [17]:
modelwise_confusion_matrices

{'RN50x64': array([[26.907, 17.797, 16.525,  0.   , 31.568,  3.39 ,  3.814],
        [ 3.109, 50.777,  0.777,  0.   ,  9.585, 12.694, 23.057],
        [ 2.649, 45.475,  9.934,  0.221, 27.373,  1.325, 13.024],
        [18.857, 26.286,  2.857,  0.   , 12.571, 37.714,  1.714],
        [10.526,  0.   ,  0.   ,  0.   , 88.158,  0.877,  0.439],
        [ 1.511,  6.344,  2.417,  0.   ,  0.302, 89.426,  0.   ],
        [12.139,  8.382,  2.601,  0.   ,  3.757, 73.121,  0.   ]]),
 'ViT-bigG-14-CLIPA-336': array([[71.61 ,  0.212, 27.966,  0.   ,  0.   ,  0.212,  0.   ],
        [12.435, 75.907,  8.29 ,  1.036,  0.518,  1.554,  0.259],
        [ 7.947, 23.62 , 67.55 ,  0.221,  0.   ,  0.   ,  0.662],
        [22.857, 10.857,  9.714,  3.429,  0.571, 52.571,  0.   ],
        [ 9.211,  0.   ,  0.877,  0.   , 89.912,  0.   ,  0.   ],
        [ 1.511,  0.   ,  0.302,  0.   ,  0.   , 98.187,  0.   ],
        [ 6.069,  1.734,  0.867,  0.   ,  0.   , 91.329,  0.   ]]),
 'ViT-L_14': array([[94.915,  0.   ,

In [18]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,modelname in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[modelname],
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)
    subplot['layout']['xaxis']['autorange'] = "reversed"

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    fig.update_xaxes(row=i//2+1, col=i%2+1, autorange='reversed')
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [19]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
8,combined,54.70
3,ViT-H-14-378-quickgelu,53.01
1,ViT-bigG-14-CLIPA-336,50.82
7,EVA02-E-14-plus,50.63
4,ViT-SO400M-14-SigLIP-384,49.64
2,ViT-L_14,42.67
5,RN101,41.28
6,ViT-B_32,36.82
0,RN50x64,33.15


In [20]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [21]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [22]:
merged_selected = pd.concat(selected_models).reset_index()

In [23]:
true_classes = merged_selected.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [24]:
merged_selected = merged_selected.groupby('image_name').sum(numeric_only=True)

In [25]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [26]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [27]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r, normalize='true') * 100
c_mat = c_mat.round(3)

In [28]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [29]:
100 * (c_mat.diagonal().sum() / (100 * len(available_models)))

56.850750000000005